## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [4]:
import urllib.request
import zipfile
import lxml.etree

In [5]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [6]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [7]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [4]:
import re
sentences_strings_ted = []
s=0
for line in open('europarl-v7.de-en.de','r',encoding='utf8').readlines():
    if s<10000:
        m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
        sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)
        s+=1
    else:
        break

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

['Wiederaufnahme der Sitzungsperiode',
 'Ich erkläre die am Freitag, dem 17',
 ' Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten',
 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten',
 ' Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden']

In [5]:
import re
sentences_strings_ted_en = []
s=0
for line in open('europarl-v7.de-en.en','r',encoding='utf8').readlines():
    if s<10000:
        m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
        sentences_strings_ted_en.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)
        s+=1
    else:
        break

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted_en[:5]

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful",
 'You have requested a debate on this subject in the course of the next few days, during this part-session',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [6]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [7]:
sentences_ted_en = []
for sent_str in sentences_strings_ted_en:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted_en.append(tokens)

Two sample processed sentences:

In [9]:
len(sentences_ted),len(sentences_ted_en)

(10878, 10654)

In [10]:
print(sentences_ted[0])
print(sentences_ted[1])

['wiederaufnahme', 'der', 'sitzungsperiode']
['ich', 'erkl', 're', 'die', 'am', 'freitag', 'dem', '17']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [11]:
counts_ted={}
for sentence in sentences_ted:
    for word in sentence:
        if word not in counts_ted:
            counts_ted[word]=1
        if word in counts_ted:
            counts_ted[word]+=1
words_ted_top1000=sorted(counts_ted, key=lambda x:int(counts_ted[x]), reverse=True)[:1000]
counts_ted_top1000=[]
for word in words_ted_top1000:
    counts_ted_top1000.append(counts_ted[word])
print(counts_ted_top1000[:10])


[10247, 9104, 6844, 4033, 3086, 3023, 2929, 2928, 2796, 2777]


In [15]:
counts_ted_en={}
for sentence in sentences_ted_en:
    for word in sentence:
        if word not in counts_ted_en:
            counts_ted_en[word]=1
        if word in counts_ted_en:
            counts_ted_en[word]+=1
words_ted_top1000_en=sorted(counts_ted_en, key=lambda x:int(counts_ted_en[x]), reverse=True)[:1000]
counts_ted_top1000_en=[]
for word in words_ted_top1000_en:
    counts_ted_top1000_en.append(counts_ted_en[word])
len(counts_ted_top1000_en)


1000

Plot distribution of top-1000 words

In [15]:
hist, edges = np.histogram(np.log(counts_ted_top1000), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [16]:
from gensim.models import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

In [21]:
model_ted_en = Word2Vec(sentences_ted_en, size=100, window=5, min_count=5, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [22]:
model_ted.most_similar("der")

[('im', 0.9507681727409363),
 ('des', 0.9460345506668091),
 ('rates', 0.9019601345062256),
 ('einer', 0.9016953706741333),
 ('eines', 0.8971198797225952),
 ('rahmen', 0.8942176699638367),
 ('zur', 0.8923423290252686),
 ('freiheit', 0.8847234845161438),
 ('sowie', 0.8827424049377441),
 ('erla', 0.881744384765625)]

In [23]:
model_ted_en.most_similar("man")

[('head', 0.9921201467514038),
 ('feeling', 0.990925669670105),
 ('paying', 0.9905898571014404),
 ('erika', 0.9905449151992798),
 ('earlier', 0.9898765683174133),
 ('positive', 0.9898198246955872),
 ('everyone', 0.9896893501281738),
 ('juris', 0.9896409511566162),
 ('mentioning', 0.9891901016235352),
 ('definition', 0.9887174367904663)]

In [30]:
import math
il=model_ted.wv['']
on=model_ted_en.wv['she']
dist=0
for i in range(100):
    curent=(il[i]-on[i])**2
    dist+=curent
dist=math.sqrt(dist)
print(dist)

5.050520998498919


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [21]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_ted=words_ted_top1000
words_top_vec_ted = model_ted[words_top_ted]

In [22]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [23]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [28]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [29]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

MemoryError: 

Preprocess sentences (note that it's important to remove small sentences for performance)

In [30]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [40]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/8)]
print(len(sentences_wiki))
sentences_wiki = [sentence.split() for sentence in sentences_wiki]

29430
3678


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [41]:
counts_wiki={}
for sentence in sentences_wiki:
    for word in sentence:
        if word not in counts_wiki:
            counts_wiki[word]=1
        if word in counts_wiki:
            counts_wiki[word]+=1
words_wiki_top1000=sorted(counts_wiki, key=lambda x:int(counts_wiki[x]), reverse=True)[:1000]
counts_wiki_top1000=[]
for word in words_wiki_top1000:
    counts_wiki_top1000.append(counts_wiki[word])
print(counts_wiki_top1000[:10])
model_wiki= Word2Vec(sentences_wiki, size=100, window=5, min_count=5, workers=4)

[3203, 2354, 1975, 1784, 1622, 1482, 1284, 1202, 1128, 1072]


#### t-SNE visualization

In [42]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_wiki_top1000]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [43]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_wiki_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)